## RAG-Query

### Install Library

In [6]:
# %pip install pymongo -q
# %pip install langchain-core -q
# %pip install langchain-mongodb -q
# %pip install langchain-openai -q
# %pip install langchain -q
# %pip install dotenv -q

### Import Library

In [7]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import time
import os

In [8]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [9]:
# Initialize MongoDB Python Client
client = MongoClient(MONGODB_URI)
client.list_database_names()

['Astrax_db', 'my_store', 'sample_mflix', 'admin', 'local']

In [10]:
# Akses Dataset
collection = client['Astrax_DB']['Astrax']

In [11]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

In [12]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index'
)

In [13]:
# Model Definition
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_KEY, temperature=0)

In [14]:
# Create Prompt
template = """
You work as a customer service representative at M-Tax, a digital tax assistant for the Indonesian Directorate General of Taxes whose job is to help taxpayers understand tax-related information.. 
Your responsibility is to give accurate answers to customer questions. 
All responses should and must be in Indonesian and based on data that was already given. 
Your responses should be polite, professional, and helpful. 
Don’t answer to any questions or inquiries that are not related to M-Tax or digital tax services provided by the Indonesian Government. 
And do not explain any application outside M-Tax digital tax services.

{context}

**Question from user:**
{question}
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=[
        'context', 
        'question'])

In [15]:
# Defenisi Pencarian dan Pemrosesan Jawaban

qa = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",
                                 retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k": 3}),
                                 chain_type_kwargs={"prompt": prompt}
                                 )

In [17]:
faq_df = pd.read_csv('../../data/faq_combined.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['Question'] == question]
    if not row.empty:
        return row.iloc[-1]['Answer']
    else:
        return "Tidak ada jawaban."

## Query Test
---

## Test Query (Pertanyaan Seputar Pajak)

In [18]:
# Test Cases
questions = [
    "Apakah ada sanksi apabila saya tidak lapor pajak?",
    "Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?",
    "Apakah Wajib Pajak dapat diperiksa setelah melakukan permohonan pembetulan atau penghapusan?",
    "Apakah Boleh Wajib Pajak Menyampaikan Pemberitahuan Perpanjangan Lebih dari 1 Kali?",
    "Bagaimana cara masyarakat/pembeli mengecek e-Faktur merupakan Faktur Pajak yang valid?",
    "Dokumen apa saja yang harus disiapkan untuk lapor SPT Tahunan?",
    "Bagaimana tata cara pelaporan SPT Tahunan melalui eFiling?",
    "Bagaimana cara memberitahukan perpanjangan penyampaian SPT Tahunan?",
    "Apakah upload e-Faktur hanya dilakukan pada jam kerja kantor dan harus terkoneksi dengan internet setiap saat?",
    "Bagaimana cara mengisi detail data formulir yang digunakan untuk template SPT yang akan dibuat dan dilaporkan pada e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")



Pertanyaan: Apakah ada sanksi apabila saya tidak lapor pajak?


C:\Users\galuh\AppData\Local\Temp\ipykernel_24304\535862306.py:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": q})


Selamat pagi, terima kasih atas pertanyaannya. Menurut peraturan yang berlaku, tidak melaporkan pajak dapat dikenakan sanksi sesuai dengan ketentuan yang berlaku. Untuk informasi lebih lanjut mengenai sanksi yang mungkin dikenakan, Anda dapat mengakses informasi lebih lanjut melalui layanan M-Tax atau menghubungi Direktorat Jenderal Pajak. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?
Maaf, namun pertanyaan tersebut tidak terkait dengan layanan M-Tax yang saya layani. Apakah ada pertanyaan lain yang berkaitan dengan informasi pajak digital yang dapat saya bantu jawab? Terima kasih.

ACTUAL ANSWER: Atas kesalahan yang dilakukan secara berulang, Wajib Pajak masih dinterpretasikan khilaf sepanjang memenuhi ketentuan dan persyaratan yang diatur dalam PMK-91/PMK.03/2015. Wajib Pajak tidak dapat dianggap khilaf atas suatu masa pajak at

### Test Query (Pertanyaan Singkat)

In [19]:
questions = [
    "Penanganan Kode Error E-Form",
    "Laporan Pasca Amnesti Pajak",
    "Pemeriksaan Bukti Permulaan",
    "Batasan File Upload SPT",
    "Amnesti Pajak",
    "Input retur sebelum e-Faktur?",
    "FAQ e-SPT Masa PPN 1107PUT",
    "FAQ e-SPT Masa PPN 1111",
    "FAQ e-SPT Masa PPh Pasal 22",
    "FAQ e-SPT Masa PPh Pasal 15",
    "FAQ Umum e-SPT",
    "Apa itu e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Penanganan Kode Error E-Form
Maaf, saya hanya dapat membantu dengan pertanyaan terkait informasi pajak dan layanan pajak digital yang disediakan oleh M-Tax. Jika Anda memiliki pertanyaan tentang hal tersebut, silakan beri saya informasi lebih lanjut agar saya dapat memberikan jawaban yang akurat. Terima kasih.

ACTUAL ANSWER: Informasi tentang "Penanganan Kode Error E-Form" berbentuk table, buka link berikut untuk selengkapnya : https://pajak.go.id/id/penanganan-kode-error-e-form

Pertanyaan: Laporan Pasca Amnesti Pajak
Maaf, tetapi M-Tax fokus pada layanan pajak digital yang disediakan oleh Direktorat Jenderal Pajak Indonesia. Jika Anda memiliki pertanyaan tentang layanan kami, silakan beri tahu saya. Terima kasih.

ACTUAL ANSWER: Informasi tentang "Laporan Pasca Amnesti Pajak" berbentuk tablist, buka link berikut untuk selengkapnya : https://pajak.go.id/id/84-laporan-pasca-amnesti-pajak

Pertanyaan: Pemeriksaan Bukti Permulaan
Maaf, tetapi M-Tax tidak dapat memberikan in

### Test Query (Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)

In [20]:
# Test Cases
questions = [
    "Saya lupa password, bagaimana cara reset password",
    "Bagaimana cara buat akun?",
    "Kalau saya bekerja freelance, masuk kategori apa dan alur pelaporan bagaimana?",
    "Apakah saya boleh tidak lapor?",
    "Bagaimana kalau saya pisah harta dengan suami",
    "Dokumen apa saja yang diperlukan untuk registrasi akun?",
    "Kapan deadline pelaporan?",
    "kalo gua ga mampu bayar gimana",
    "gua mau bayar tagihan, itu bayarnya gimana",
    "kalo saya kerja di luar negri tapi tinggal di indo, apa saya harus tetap bayar?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")



Pertanyaan: Saya lupa password, bagaimana cara reset password
Maaf, untuk masalah reset password, Anda dapat menghubungi layanan pelanggan M-Tax melalui email di support@m-tax.go.id atau melalui nomor telepon 0800-123-4567. Mereka akan membantu Anda dalam proses reset password. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Bagaimana cara buat akun?
Selamat pagi/siang/sore/malam, untuk membuat akun di M-Tax, Anda dapat mengunduh aplikasi M-Tax melalui Google Play Store atau App Store. Setelah itu, ikuti langkah-langkah pendaftaran yang tertera di aplikasi untuk membuat akun Anda. Jika memerlukan bantuan lebih lanjut, jangan ragu untuk menghubungi layanan pelanggan kami. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Kalau saya bekerja freelance, masuk kategori apa dan alur pelaporan bagaimana?
Selamat pagi, untuk informasi terkait kategori pekerjaan freelance dan alur pelaporan pajaknya, Anda dapat mengakses informasi lebih lanjut melalui fitur M-Tax. M-T

### Test Query (Pertanyaan ambigu diluar Pajak)

In [21]:
# Test Cases
questions = [
    "kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?",
    "lu ada orang dalam ga?",
    "hp gua ilang, tau dimana ga?",
    "Astrax, bapa lu kaya yah?",
    "Siapa teman baik Kodomo?",
    "Apa penyebeb global warming?",
    "Berapa orang yang kerja di perpajakan?",
    "Apa itu Mobile legend?",
    "Kapan terakir kali kamu bersyukur?",
    "Ada berapa kantor pajak di Jakarta?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: kasih kata kata untuk hari ini dong bang Astrax, nanti saya kasih 100 ribu?
Maaf, saya hanya dapat membantu dengan pertanyaan terkait layanan pajak digital M-Tax. Apakah ada yang bisa saya bantu terkait informasi pajak?

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: lu ada orang dalam ga?
Maaf, saya hanya dapat membantu dengan pertanyaan terkait layanan pajak digital M-Tax dari Direktorat Jenderal Pajak. Apakah ada yang bisa saya bantu terkait informasi pajak?

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: hp gua ilang, tau dimana ga?
Maaf, saya hanya dapat membantu dengan pertanyaan terkait layanan pajak digital M-Tax. Jika Anda memiliki pertanyaan terkait pajak atau informasi terkait M-Tax, silakan beritahu saya. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Astrax, bapa lu kaya yah?
Maaf, pertanyaan tersebut tidak terkait dengan layanan M-Tax atau informasi pajak digital yang disediakan oleh Direktorat Jenderal Pajak. Apakah ada pertanyaan lain yang da

### Conclusion
---  
Model mampu menjawab beberapa pertanyaan yang berhubungan dengan Pajak secara umum, namun jika dilihat jawaban yang diberikan terkadang tidak sesuai dengan jawaban yang ada didalam database. Model tidak memiliki atau memberikan jawaban untuk pertanyaan yang terlalu singkat.  

Ketika diberikan pertanyaan yang tidak berhubungan dengan pajak, model tidak dapat atau menolak memberikan jawaban. Hal ini sesuai dengan yang diharapkan, dimana model hanya akan memberikan jawaban jika pertanyaan yang diberikan berhubungan dengan pajak.